In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import confusion_matrix

import numpy as np
from sklearn.metrics import precision_score

In [3]:
df=pd.read_csv("/Users/elianezanlorense/Downloads/winequality.csv",sep=";")

In [10]:
df.head()

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,quality,alcohol
0,White,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,6,8.8
1,White,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,6,9.5
2,White,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,6,10.1
3,White,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,6,9.9
4,White,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,6,9.9


In [9]:
df.dtypes

type                     object
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
quality                   int64
alcohol                 float64
dtype: object

In [6]:
#Como a variavel alcohol esta como objeto e nao em um formato numerico, 
#como por exemplo float ou int,a variavel será transformada para numeric
alcool_new=pd.to_numeric(df['alcohol'], errors='coerce').fillna(0)

In [7]:
#dropando a variavel alcohol e substituindo pela variavel float
df=df.drop('alcohol',axis=1)
df=pd.concat([df,alcool_new],axis=1)

In [8]:
#todas as variaveis estao no formato correto, a variavel 'type' será dumizada posteriormente
df.dtypes

type                     object
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
quality                   int64
alcohol                 float64
dtype: object

In [ ]:
#conferindo a volumetria e o numero de variaveis
df.shape

In [ ]:
keep=['pH','fixed acidity','total sulfur dioxide','volatile acidity','type']
df_1=df[keep]
sns.pairplot(df_1, hue = 'type', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)

In [ ]:
df_2 = sns.boxplot(y='pH', x='quality',
                 data=df, 
                 width=0.5,
                 palette="colorblind")


In [ ]:
df.dtypes

In [ ]:
df.sort_values(['quality'])

In [ ]:
df['quality'].plot(kind='bar')
plt.xlabel("Quality", labelpad=14)

In [ ]:
#Listando todas as variaveis categoricas para torna-las dummies
dummies=[
   'type'
]
dummies=pd.get_dummies(df[dummies], drop_first=True)
dummies.shape
dummies.head()

In [ ]:
#dropando a variavel 'type'que foi categorizada acima, em seguida agregando a variavel 'type_white'
df=df.drop('type',axis=1)

In [ ]:
df=pd.concat([df, dummies],axis=1)
df.shape
#df.head()
df.dtypes

In [ ]:
#verificando se existe alguma variavel missing, caso positivo seria corrigida 
df.isnull().sum()

In [ ]:
#após feito todas as correcoes, iremos definir a amostra de teste e amostra de validacao
df.head()
df.shape
y=df['quality']
y.head()
y.shape
X=df.drop(['quality'],axis=1)

In [ ]:
X.dtypes

In [ ]:
X.head()

In [ ]:
#Divindo a amostra em validacao e treino, neste caso usou-se a proporcao 70/30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [ ]:
X_train.shape
y_train.shape
X_train.head()

In [ ]:
X_cor=X_train.drop(['type_White'],axis=1)

In [ ]:
#Verificando a variancia, baixa variabilidade será removida
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(X_cor)
len(X_cor.columns[constant_filter.get_support()])
constant_columns = [column for column in X_cor.columns
                    if column not in X_cor.columns[constant_filter.get_support()]]

print(len(constant_columns))
for column in constant_columns:
    print(column)
#Neste caso todas as variaveis permanecem

In [ ]:
#Variaveis altamente correlacionadas serao removidas, neste caso o corte considerado foi +- 0.7
correlated_features = set()
correlation_matrix =X_cor.corr()
for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
len(correlated_features)
print(correlated_features)

#Nota-se que a varialve total sulfur dioxide apresenta correlacao superio a 0.7, e será removida
#para a construcao do modelo.

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(correlation_matrix, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Remoevendo da amostra do treino, as variaveis altamente correlacionadas, 
#que neste caso foi 'total sulfur dioxide'
X_train=X_train.drop(labels=correlated_features, axis=1)
X_train.head()
X_train.shape
select_vars=list(X_train)
X_test=X_test[select_vars]

In [ ]:
#Aplicando a tecnica GradienteBoostingClassifier, sem nenhuma otimizacao de parametros
#Ou seja todos os parametro estao sendo usados os seus valores default do algoritmo
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

In [ ]:
#Aplicando na amostra de validacao, os parametros otimizados acima, ou seja, com os valores
#otimizados, iremos prever a qualidade de vinho em uma parte da populcao (30%)
y_pred =model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
#Importancia das variaveis
sorted_idx = np.argsort(model.feature_importances_)[::-1]
for index in sorted_idx:
    a=([X_train.columns[index], model.feature_importances_[index]]) 
    print(a)
plt.bar(range(len(model.feature_importances_)), model.feature_importances_ )

In [ ]:
#Para tentar melhorar a perfomance, vamos aplicar grdsearch, isto a combinaocao de varios valores dos parametros
lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

for learning_rate in lr_list:
    gb_clf = GradientBoostingClassifier(n_estimators=20, learning_rate=learning_rate)
    gb_clf.fit(X_train, y_train)

    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

In [ ]:
#Analisando o accuracy no treino e validacao, obtemos accuract alto no traino, 
#mas na validacao estes valores caem, isto indica que o modelo esta instável.
#A melhor parametrizacao neste caso é considerando Learnin rate:0.5
#Neste caso consideramos o primeiro modelo.
model_2 = GradientBoostingClassifier(n_estimators=20, learning_rate=0.05)
model_2.fit(X_train, y_train)
y_pred_2=model_2.predict(X_test)
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_2))

In [ ]:
#Utilizando a library xgboos (anteriormente estavamso usando xgboost skleanr)
#Umas das vantagens de utilizar este pacote é, mais pode de parametrizacao
def prec_xgb(n_trees, max_depth, X_train, y_train, X_test, y_test, learning_rate=0.005):
    """
    ExtraTrees
    """
    import xgboost as xgb
  
    LOGGER.info('start predict: n_trees={},X_train.shape={},y_train.shape={},X_test.shape={},y_test.shape={}'.format(
        n_trees, X_train.shape, y_train.shape, X_test.shape, y_test.shape))
    model_2 = xgb.XGBClassifier(n_estimators=n_trees, max_depth=2, 
                            objective='multi:softprob',sample_type='weighted',
            seed=123, silent=True, nthread=-1, learning_rate=learning_rate)
    eval_set = [(X_test, y_test)]
    model_2.fit(X_train, y_train, eval_set=eval_set, eval_metric="merror")
    y_pred = clf.predict(X_test)
    prec = float(np.sum(y_pred == y_test)) / len(y_test)
    LOGGER.info('prec_xgb_{}={:.6f}%'.format(n_trees, prec*100.0))
    return clf, y_pred 
print(confusion_matrix(y_test, y_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))